In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
!lscpu | grep 'Model name'
!lscpu | grep 'Socket(s):'
!lscpu | grep 'Core(s) per socket'
!lscpu | grep 'Thread(s) per core'
!nvidia-smi

import numpy as np
import tensorflow as tf
import warnings

from keras.layers import Input, Dropout, Dense, Flatten, Activation, MaxPooling2D, Conv2D, AveragePooling2D, BatchNormalization, concatenate
from keras.initializers import he_normal
from keras.regularizers import l2
from keras.models import Model

from keras import backend as K

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64

ds_train = tf.keras.utils.image_dataset_from_directory(
    directory = "../input/imagenetmini-1000/imagenet-mini/train",
    labels = "inferred",
    label_mode = "int",
    color_mode = "rgb",
    batch_size = BATCH_SIZE,
    image_size = (299, 299),
)

ds_val = tf.keras.utils.image_dataset_from_directory(
    directory = "../input/imagenetmini-1000/imagenet-mini/val",
    labels = "inferred",
    label_mode = "int",
    color_mode = "rgb",
    batch_size = BATCH_SIZE,
    image_size = (299, 299)
)

def normalize_image(image, label):
    #image = tf.keras.applications.resnet.preprocess_input(image)
    image = tf.cast(x=image, dtype=tf.float32)/127.5 - 1.0
    return image, label

#ds_train = ds_train.shuffle(100)
ds_train = ds_train.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
#ds_train = ds_train.cache()
#ds_train = ds_train.prefetch(buffer_size=AUTOTUNE)

ds_val = ds_val.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.prefetch(buffer_size=AUTOTUNE)

Inception Modules

In [ ]:
def conv_bn(x, filters, kernel_size, strides, padding='same', use_bias=False):
    # Define the joint function to apply the combination of Conv and BatchNorm 
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias,
               kernel_initializer="he_normal", kernel_regularizer=l2(0.00004))(x)
    x = BatchNormalization(axis=3, momentum=0.9997, scale=False)(x)
    x = Activation('relu')(x)

    return x


def inception_stem(input):
    # Define the stem network 
    stem = conv_bn(input, filters=32, kernel_size=(3,3), strides=(2,2), padding='valid')
    stem = conv_bn(stem, filters=32, kernel_size=(3,3), strides=(1,1), padding='valid')
    stem = conv_bn(stem, filters=64, kernel_size=(3,3), strides=(1,1))

    branch_11 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(stem)
    branch_12 = conv_bn(stem, filters=96, kernel_size=(3,3), strides=(2,2), padding='valid')
    
    stem = concatenate([branch_11, branch_12], axis=3)

    branch_13 = conv_bn(stem, filters=64, kernel_size=(1,1), strides=(1,1))
    branch_14 = conv_bn(branch_13, filters=96, kernel_size=(3,3), strides=(1,1), padding='valid')
    branch_15 = conv_bn(stem, filters=64, kernel_size=(1,1), strides=(1,1))
    branch_16 = conv_bn(branch_15, filters=64, kernel_size=(1,7), strides=(1,1))
    branch_17 = conv_bn(branch_16, filters=64, kernel_size=(7,1), strides=(1,1))
    branch_18 = conv_bn(branch_17, filters=96, kernel_size=(3,3),  strides=(1,1), padding='valid')
    
    stem = concatenate([branch_14,branch_18], axis=3)

    branch_19 = conv_bn(stem, filters=192, kernel_size=(3,3), strides=(2,2), padding='valid')
    branch_20 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(stem)
    
    x = concatenate([branch_19,branch_20], axis=3)
  
    return x


def inception_a(input):

    branch_11 = conv_bn(input, filters=96, kernel_size=(1,1), strides=(1,1))

    branch_12 = conv_bn(input, filters=64, kernel_size=(1,1), strides=(1,1) )
    branch_22 = conv_bn(branch_12, filters=96, kernel_size=(3,3), strides=(1,1))

    branch_13 = conv_bn(input, filters=64, kernel_size=(1,1), strides=(1,1))
    branch_23 = conv_bn(branch_13, filters=96, kernel_size=(3,3), strides=(1,1))
    branch_33 = conv_bn(branch_23, filters=96, kernel_size=(3,3), strides=(1,1))

    branch_14 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_24 = conv_bn(branch_14, filters=96, kernel_size=(1,1), strides=(1,1))

    x = concatenate([branch_11,branch_22,branch_33,branch_24], axis=3)
    
    return x


def reduction_a(input):

    branch_11 = conv_bn(input, filters=384, kernel_size=(3,3), strides=(2,2), padding='valid')

    branch_12 = conv_bn(input, filters=192, kernel_size=(1,1), strides=(1,1))
    branch_22 = conv_bn(branch_12, filters=224, kernel_size=(3,3), strides=(1,1))
    branch_32 = conv_bn(branch_22, filters=256, kernel_size=(3,3), strides=(2,2), padding='valid')

    branch_13 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(input)

    x = concatenate([branch_11,branch_32,branch_13], axis=3)

    return x


def inception_b(input):

    branch_11 = conv_bn(input, filters=384, kernel_size=(1,1), strides=(1,1))

    branch_12 = conv_bn(input, filters=192, kernel_size=(1,1), strides=(1,1))
    branch_22 = conv_bn(branch_12, filters=224, kernel_size=(1,7), strides=(1,1))
    branch_32 = conv_bn(branch_22, filters=256, kernel_size=(7,1), strides=(1,1))

    branch_13 = conv_bn(input, filters=192, kernel_size=(1,1), strides=(1,1))
    branch_23 = conv_bn(branch_13, filters=192, kernel_size=(7,1), strides=(1,1))
    branch_33 = conv_bn(branch_23, filters=224, kernel_size=(1,7), strides=(1,1))
    branch_43 = conv_bn(branch_33, filters=224, kernel_size=(7,1), strides=(1,1))
    branch_53 = conv_bn(branch_43, filters=256, kernel_size=(1,7), strides=(1,1))

    branch_14 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_24 = conv_bn(branch_14, filters=128, kernel_size=(1,1), strides=(1,1))

    x = concatenate([branch_11,branch_32,branch_53,branch_24], axis=3)

    return x


def reduction_b(input):

    branch_11 = conv_bn(input, filters=192, kernel_size=(1,1), strides=(1,1))
    branch_21 = conv_bn(branch_11, filters=192, kernel_size=(3,3), strides=(2,2), padding='valid')

    branch_12 = conv_bn(input, filters=256, kernel_size=(1,1), strides=(1,1))
    branch_22 = conv_bn(branch_12, filters=256, kernel_size=(1,7), strides=(1,1))
    branch_32 = conv_bn(branch_22, filters=320, kernel_size=(7,1), strides=(1,1))
    branch_42 = conv_bn(branch_32, filters=320, kernel_size=(3,3), strides=(2,2), padding='valid')

    branch_13 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(input)

    x = concatenate([branch_21,branch_42,branch_13], axis=3)

    return x


def inception_c(input):

    branch_11 = conv_bn(input, filters=256, kernel_size=(1,1), strides=(1,1))

    branch_12 = conv_bn(input, filters=384, kernel_size=(1,1), strides=(1,1))
    branch_22 = conv_bn(branch_12, filters=256, kernel_size=(1,3), strides=(1,1))
    branch_23 = conv_bn(branch_12, filters=256, kernel_size=(3,1), strides=(1,1))
    branch_33 = concatenate([branch_22,branch_23], axis=3)

    branch_14 = conv_bn(input, filters=384, kernel_size=(1,1), strides=(1,1))
    branch_24 = conv_bn(branch_14, filters=448, kernel_size=(3,1), strides=(1,1))
    branch_34 = conv_bn(branch_24, filters=512, kernel_size=(1,3), strides=(1,1))
    branch_44 = conv_bn(branch_34, filters=256, kernel_size=(1,3), strides=(1,1))
    branch_45 = conv_bn(branch_34, filters=256, kernel_size=(3,1), strides=(1,1))
    branch_55 = concatenate([branch_44,branch_45], axis=3)

    branch_16 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_26 = conv_bn(branch_16, filters=256, kernel_size=(1,1), strides=(1,1))

    x = concatenate([branch_11,branch_33,branch_55,branch_26], axis=3)

    return x


def inception_v4(input_shape, num_classes, include_top):
    # Build the abstract Inception v4 network
    '''
    Args:
        input_shape: three dimensions in the TensorFlow Data Format
        num_classes: number of classes
        weights: pre-defined Inception v4 weights 
        include_top: a boolean, for full traning or finetune 
    Return: 
        logits: the logit outputs of the model.
    '''
    inputs = Input(shape=input_shape)

    # Make the the stem of Inception v4 
    x = inception_stem(inputs)

    # 4 x Inception-A blocks: 35 x 35 x 384
    for i in range(0, 4):
        x = inception_a(x)

    # Reduction-A block: # 35 x 35 x 384
    x = reduction_a(x)

    # 7 x Inception-B blocks: 17 x 17 x 1024
    for i in range(0, 7):
        x = inception_b(x)

    # Reduction-B block: 17 x 17 x 1024
    x = reduction_b(x)

    # 3 x Inception-C blocks: 8 x 8 x 1536
    for i in range(0, 3):
        x = inception_c(x)

    # Final pooling and prediction
    if include_top:
        # 1 x 1 x 1536
        x = AveragePooling2D((8,8), padding='valid')(x)
        x = Dropout(0.5)(x)
        x = Flatten()(x)
        x = Dense(units=num_classes, activation=None)(x)

    model = Model(inputs, x, name='inception_v4')
    return model


Inception_V4

In [ ]:
input_shape = (299,299,3)
num_classes = 1000
include_top = True 

model = inception_v4(input_shape, num_classes, include_top)
    
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
)

model.summary()

In [ ]:
checkpoint_filepath = 'best_ImageNet_V4.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='"val_sparse_top_k_categorical_accuracy"',
    mode='max',
    save_best_only=True)
model.fit(ds_train, validation_data=ds_val, verbose=1, epochs=200, callbacks=[model_checkpoint_callback])

In [ ]:
model.evaluate(ds_val, verbose=1)

In [ ]:
!pip install keras-flops
from keras_flops import get_flops
flops = get_flops(model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

In [ ]:
import os
Model_Name = "InceptionV4_50ep_24_6GF"
os.environ["MODELNAME"] = Model_Name
os.environ["MODELNAMEZIP"] = Model_Name + ".zip"
model.save("./" + Model_Name)
!zip -r ./$MODELNAMEZIP ./$MODELNAME
!rm -rf ./$MODELNAME